In [1]:
# start with same libraries as vector tutorial
%matplotlib inline

from __future__ import (absolute_import, division, print_function)
import os
import sys
import json
import psycopg2

import matplotlib as mpl
import matplotlib.pyplot as plt
import folium
from folium.plugins import TimeSliderChoropleth

from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

/root/miniconda/envs/backend/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
%run -i load_data.py

# Lets view our data from the Excel File!

In [3]:
co2_geo_data_table.head(1)

,Reference,Site Type,Latitude,Longitude,DateTime,Altitude,Discharge,Stream Slope,Width,Depth,...,Air Temperature,DO,Chl a,CO2,k,k600,CO2 Flux,CO2 Flux.1,Source,geometry
0,"Alin and Richey, 2012",small stream,-10.066,-67.606,20040701,150.0,NaN,NaN,NaN,NaN,...,28.1,NaN,NaN,140.5,NaN,-18.104,1381.28,NaN,"Alin, S.R., and J.E. Richey. 2012. LBA-ECO CD-...",POINT (-67.60599999999999 -10.066)


We are going to select all the unique locations from the Exel file, drop any columns where there are null values and reset the index to create a new clean locations dataframe.

In [4]:
co2_data_locations = co2_geo_data_table[["Latitude", "Longitude", "Altitude", "Site Type"]].drop_duplicates()
good_co2_data_locations = co2_data_locations.dropna()
co2_data_locations_clean = good_co2_data_locations.reset_index(drop='index')
co2_data_locations_clean.head(5)
len(co2_data_locations_clean)


183

We are going to transform our dataframe above into a Geodataframe which will allow us to spaitally join our locations back to out samples. We are also creating a new column "location_id" which is going to be the index of our locations when we load them into the postgis db.

In [5]:
geometry = [Point(xy) for xy in zip(co2_data_locations_clean['Longitude'], co2_data_locations_clean['Latitude'])]
geometry = GeoSeries(geometry)
geometry.crs = {'init': 'epsg:4326'}
co2_geometries = GeoDataFrame(co2_data_locations_clean, geometry=geometry, crs=geometry.crs)
co2_geometries
co2_geometries.loc[:, 'location_id'] = co2_geometries.index

In [19]:
co2_geometries.head(3)

,Latitude,Longitude,Altitude,Site Type,geometry,location_id
0,-10.066,-67.606,150.0,small stream,POINT (-67.60599999999999 -10.066),0
1,-9.751,-67.672,150.0,small stream,POINT (-67.672 -9.750999999999999),1
2,-9.016,-68.584,150.0,small river,POINT (-68.584 -9.016),2


Join our locations back to the samples geodataframe where the "Sample Point Geometry" equals the "Location Point Geometry"

In [7]:

samples = gpd.sjoin(co2_geo_data_table, co2_geometries, how="inner", op='intersects')

In [8]:
samples[["DateTime", "CO2", "location_id"]].head(10)

,DateTime,CO2,location_id
0,20040701,140.5,0
1,20040701,140.5,0
2,20040701,140.5,0
3,20040701,140.5,0
4,20040701,140.5,0
256,20050830,2662.2,0
257,20050830,2662.2,0
258,20050830,2662.2,0
259,20050830,2662.2,0
0,20040701,140.5,63


Add the Sample Type CO2 with Unit 'uatm' since we are only currenlty loading CO2 observations and clean the joined sample data to prepare to load into postgis

In [9]:
samples["SampleType"] = 'CO2'
samples["Unit"] = 'uatm'

good_samples = samples[["DateTime", "SampleType", "Unit", "CO2", "location_id"]].dropna()
clean_samples = good_samples[["DateTime", "SampleType", "Unit", "CO2", "location_id"]].reset_index(drop='index')
clean_samples["Index"] = clean_samples.index
clean_samples.head(10)

,DateTime,SampleType,Unit,CO2,location_id,Index
0,20040701,CO2,uatm,140.5,0,0
1,20040701,CO2,uatm,140.5,0,1
2,20040701,CO2,uatm,140.5,0,2
3,20040701,CO2,uatm,140.5,0,3
4,20040701,CO2,uatm,140.5,0,4
5,20050830,CO2,uatm,2662.2,0,5
6,20050830,CO2,uatm,2662.2,0,6
7,20050830,CO2,uatm,2662.2,0,7
8,20050830,CO2,uatm,2662.2,0,8
9,20040701,CO2,uatm,140.5,63,9


In [10]:
# cast all the datetime column to datetime type 
# Dirty data so we can't apply date_time to incorrect str formats
# samples.apply(lambda row:pd.to_datetime(row["DateTime"]), axis=1)

## Set up our DB Connection

Read DB configuration and use to connect to database. Initialize cursor in the DB.

In [11]:
with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), "db.json")) as f:
    db_conn_dict = json.load(f)
    
conn = psycopg2.connect(**db_conn_dict)
cur = conn.cursor()


Create string template of SQL procedure to load the sitelocations into the database

In [12]:
insert_locations_string = """
    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values ({0}, LEFT('{1}', 150), {2}, {3}, {4}, ST_SetSRID(ST_GeomFromText('{5}'), 4326))
"""

In [13]:
co2_data_locations_clean[["location_id", "Site Type", "Latitude", "Longitude", "Altitude", "geometry"]].head(3)

,location_id,Site Type,Latitude,Longitude,Altitude,geometry
0,0,small stream,-10.066,-67.606,150.0,POINT (-67.60599999999999 -10.066)
1,1,small stream,-9.751,-67.672,150.0,POINT (-67.672 -9.750999999999999)
2,2,small river,-9.016,-68.584,150.0,POINT (-68.584 -9.016)


Iterate through the dataframe to load all the points into the DB finally!!!

In [14]:
for index, row in co2_data_locations_clean[["location_id", "Site Type", "Latitude", "Longitude", "Altitude", "geometry"]].dropna().iterrows():
        insert_locations_string.format(row["location_id"], row["Site Type"], row["Latitude"], row["Longitude"], row["Altitude"], row["geometry"])
        print(insert_locations_string.format(row["location_id"], row["Site Type"], row["Latitude"], row["Longitude"], row["Altitude"], row["geometry"]))
        cur.execute(insert_locations_string.format(row["location_id"], row["Site Type"], row["Latitude"], row["Longitude"], row["Altitude"], row["geometry"]))
        conn.commit()


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (0, LEFT('small stream', 150), -10.066, -67.606, 150.0, ST_SetSRID(ST_GeomFromText('POINT (-67.60599999999999 -10.066)'), 4326))


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (1, LEFT('small stream', 150), -9.751, -67.672, 150.0, ST_SetSRID(ST_GeomFromText('POINT (-67.672 -9.750999999999999)'), 4326))


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (2, LEFT('small river', 150), -9.016, -68.584, 150.0, ST_SetSRID(ST_GeomFromText('POINT (-68.584 -9.016)'), 4326))


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (3, LEFT('small river', 150), -9.029, -68.593, 150.0, ST_SetSRID(ST_GeomFromText('POINT (-68.593 -9.029)'), 4326))


    INSERT INTO co2data_sitelocation (id, site_description, lati


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (73, LEFT('rivers less than 100 m wide', 150), -10.813, -62.119, 180.0, ST_SetSRID(ST_GeomFromText('POINT (-62.119 -10.813)'), 4326))


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (74, LEFT('rivers less than 100 m wide', 150), -10.75, -62.17, 180.0, ST_SetSRID(ST_GeomFromText('POINT (-62.17 -10.75)'), 4326))


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (75, LEFT('rivers less than 100 m wide', 150), -10.954, -62.256, 180.0, ST_SetSRID(ST_GeomFromText('POINT (-62.256 -10.954)'), 4326))


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (76, LEFT('rivers less than 100 m wide', 150), -10.969, -62.269, 180.0, ST_SetSRID(ST_GeomFromText('POINT (-62.269 -10.969)'), 4326))


    INSERT INTO co2


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (149, LEFT('river, tributary', 150), -15.604, 30.4103, 327.0, ST_SetSRID(ST_GeomFromText('POINT (30.4103 -15.604)'), 4326))


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (150, LEFT('river, tributary', 150), -13.932, 29.1316, 1064.0, ST_SetSRID(ST_GeomFromText('POINT (29.1316 -13.932)'), 4326))


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (151, LEFT('river, tributary', 150), -14.7943, 29.6292, 440.0, ST_SetSRID(ST_GeomFromText('POINT (29.6292 -14.7943)'), 4326))


    INSERT INTO co2data_sitelocation (id, site_description, latitude, longitude, altitude, point) 
    values (152, LEFT('river, tributary', 150), -12.2604, 26.7923, 1271.0, ST_SetSRID(ST_GeomFromText('POINT (26.7923 -12.2604)'), 4326))


    INSERT INTO co2data_sitelocation (id, site_de

Another template to load the data into Samples now.

In [15]:
insert_samples_string = """
    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values ({0}, '{1}', '{2}', '{3}', {4}, {5})
"""

In [16]:
clean_samples.head(1)

,DateTime,SampleType,Unit,CO2,location_id,Index
0,20040701,CO2,uatm,140.5,0,0


Iterate through the dataframe to load all the samples into the DB finally!!!

In [17]:
for index, row in clean_samples.iterrows():
        insert_samples_string.format(row["Index"], row["DateTime"], row["SampleType"], row["Unit"], row["CO2"], row["location_id"])
        print(insert_samples_string.format(row["Index"], row["DateTime"], row["SampleType"], row["Unit"], row["CO2"], row["location_id"]))
        cur.execute(insert_samples_string.format(row["Index"], row["DateTime"], row["SampleType"], row["Unit"], row["CO2"], row["location_id"]))
        conn.commit()


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (0, '20040701', 'CO2', 'uatm', 140.5, 0)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (1, '20040701', 'CO2', 'uatm', 140.5, 0)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (2, '20040701', 'CO2', 'uatm', 140.5, 0)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (3, '20040701', 'CO2', 'uatm', 140.5, 0)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (4, '20040701', 'CO2', 'uatm', 140.5, 0)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (5, '20050830', 'CO2', 'uatm', 2662.2, 0)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (6, '2005083


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (87, '20040722', 'CO2', 'uatm', 4247.2, 11)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (88, '20040722', 'CO2', 'uatm', 4247.2, 11)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (89, '20040810', 'CO2', 'uatm', 936.1, 12)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (90, '20040811', 'CO2', 'uatm', 503.0, 12)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (91, '20040811', 'CO2', 'uatm', 394.3, 12)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (92, '20040816', 'CO2', 'uatm', 506.9, 12)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (191, '20050309', 'CO2', 'uatm', 2904.7, 40)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (192, '20050309', 'CO2', 'uatm', 2904.7, 40)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (193, '20050309', 'CO2', 'uatm', 2904.7, 40)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (194, '20050309', 'CO2', 'uatm', 4762.2, 41)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (195, '20050309', 'CO2', 'uatm', 4762.2, 41)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (196, '20050309', 'CO2', 'uatm', 4762.2, 41)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 



    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (284, '20050904', 'CO2', 'uatm', 1110.4, 65)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (285, '20061125', 'CO2', 'uatm', 2416.052891, 67)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (286, '20061125', 'CO2', 'uatm', 2416.052891, 67)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (287, '20061125', 'CO2', 'uatm', 2416.052891, 67)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (288, '20061127', 'CO2', 'uatm', 3917.523126, 68)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (289, '20061127', 'CO2', 'uatm', 3917.523126, 68)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measure


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (377, '2012', 'CO2', 'uatm', 1056.0, 122)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (378, '41287', 'CO2', 'uatm', 2005.0, 122)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (379, '2012', 'CO2', 'uatm', 2444.0, 123)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (380, '41288', 'CO2', 'uatm', 2576.0, 123)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (381, '2012', 'CO2', 'uatm', 1968.0, 124)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (382, '41290', 'CO2', 'uatm', 2560.0, 124)


    INSERT INTO co2data_sample (id, date, sample_type, unit, measurement, site_location_id) 
    values (383

Delete Cursur, Close DB Connection, AND FINISHED!!!

In [18]:
cur.close()
conn.close()